## Selenium

Selenium est une bibliothèque Python et un outil utilisé pour automatiser les navigateurs Web afin d'effectuer un certain nombre de tâches. L’un d’entre eux est le web-scraping pour extraire des données et des informations utiles qui autrement ne seraient pas disponibles. Voici un guide étape par étape sur la façon d'utiliser Selenium, l'exemple étant l'extraction des données sur les salaires des joueurs NBA du site Web [hoopshype](https://hoopshype.com/salaries/players/).

### Etape 1 : Installation de Selenium et importation des bibliothèques

```python
!pip install selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
```

Vous avez également la possibilité de récupérer les librairies dans le dossier <strong>./packages/requirements_scrapping.txt</strong>

### Etape 2 : Installer et accéder à Webdriver

Un Webdriver est un ingrédient essentiel de ce processus. C’est ce qui ouvrira automatiquement votre navigateur pour accéder au site Web de votre choix. Cette étape est différente selon le navigateur que vous utilisez pour explorer Internet. Il se trouve que j'utilise Google Chrome. Certains disent que Chrome fonctionne mieux avec Selenium, bien qu'il prenne également en charge Internet Explorer, Firefox, Safari et Opera. Pour Chrome, vous devez d'abord télécharger le [pilote Web](https://chromedriver.chromium.org/downloads). Il existe plusieurs options de téléchargement différentes en fonction de votre version de Chrome. Pour localiser la version de Chrome dont vous disposez, cliquez sur les 3 points verticaux dans le coin supérieur droit de la fenêtre de votre navigateur, faites défiler vers le bas pour obtenir de l'aide et sélectionnez « À propos de Google Chrome ». Là, vous verrez votre version. J'ai la version 80.0.3987.149, illustrée dans les captures d'écran ci-dessous.

![image](./documentation/image1.png)

![image](./documentation/image2.png)

Vous devez maintenant savoir où vous avez enregistré le téléchargement de votre pilote Web sur votre ordinateur local. Le mien vient d'être enregistré dans mon dossier de téléchargements par défaut. Vous pouvez maintenant créer une variable de pilote en utilisant le chemin direct de l'emplacement de votre pilote Web téléchargé.

```python
driver = webdriver.Chrome('/Users/MyUsername/Downloads/chromedriver')
```

### Etape 3 : Accéder au site Web via Python

Étape très simple mais très importante. Vous avez besoin de votre code pour ouvrir le site Web que vous essayez de scrapper.

```python
driver.get('https://hoopshype.com/salaries/players/')
```

Une fois exécuté, cet extrait de code ouvrira le navigateur sur le site Web de votre choix.

### Etape 4 : Localisez les informations spécifiques que vous récupérez

Afin d’extraire les informations que vous souhaitez récupérer, vous devez localiser le XPath de l’élément. Un XPath est une syntaxe utilisée pour rechercher n'importe quel élément sur une page Web. Pour localiser le XPath de l'élément, mettez en surbrillance le premier dans la liste de ce que vous recherchez, faites un clic droit et sélectionnez inspecter ; cela ouvre les outils de développement. Pour mon exemple, je souhaite d’abord localiser les noms des joueurs NBA, je sélectionne donc d’abord Stephen Curry.

![image](./documentation/image3.png)

![image](./documentation/image4.png)

Dans les outils de développement, on voit désormais l’élément « Stephen Curry » apparaître comme tel.

![image](./documentation/image5.png)

Cet élément peut facilement être traduit en son XPath, mais d’abord, nous devons nous rappeler que nous n’essayons pas seulement de localiser cet élément, mais tous les noms de joueurs. En utilisant le même processus, j'ai localisé l'élément suivant dans la liste, Russell Westbrook.

![image](./documentation/image6.png)

Le point commun entre ces deux (et tous les autres noms de joueurs) est ```<td class="name">```, c'est donc ce que nous utiliserons pour créer une liste de tous les noms de joueurs. Cela traduit en XPath ressemble à ```//td[@class="name"]```. En décomposant cela, tous les XPaths sont précédés de la double barre oblique, que nous voulons dans une balise td, chaque classe de cette balise td devant correspondre à « nom ». Nous pouvons maintenant créer la liste des noms de joueurs avec cette fonction Selenium.

```python
players = driver.find_elements_by_xpath('//td[@class="name"]')
```

Et maintenant, pour mettre le texte de chaque nom de joueur dans une liste, nous écrivons cette fonction.

```python
players_list = []
for p in range(len(players)):
	players_list.append(players[p].text)
```

En suivant ce même processus pour acquérir les salaires des joueurs…

![image](./documentation/image7.png)

![image](./documentation/image8.png)

![image](./documentation/image9.png)

![image](./documentation/image10.png)

Lors de l'inspection de ces éléments et de la traduction vers XPath, nous pouvons ignorer le style et la valeur des données, en nous souciant uniquement de la classe.

```python
salaries = driver.find_elements_by_xpath('//td[@class="hh-salaries-sorted"]')
```

Et le texte de la liste des salaires…

```python
salaries_list = []
for s in range(len(salaries)):
	salaries_list.append(salaries[s].text)
```

### Etape 5 : Appliquez pour chaque année disponible et liez le tout ensemble

Souvent, lorsque vous utilisez Selenium, vous tenterez de récupérer des données situées sur plusieurs pages différentes du même site Web. Dans mon exemple, [hoopshype.com](hoopshype.com) dispose de données sur les salaires de la NBA remontant à la saison 1990/91.

![image](./documentation/image11.png)
![image](./documentation/image12.png)

Comme vous pouvez le constater, la différence entre l'URL de chaque saison est simplement une question d'années incluses à la fin. Ainsi, l'URL de la saison 2018/19 est ```https://hoopshype.com/salaries/players/2018-2019/``` et l'URL de la saison 1990/91 est ```https://hoopshype.com/salaries/players/1990-1991/```. Avec cela, nous pouvons créer une fonction qui parcourt chaque année et accède à chaque URL individuellement, puis regroupe toutes les étapes que nous avons précédemment présentées pour chaque année individuellement. J'associe également chaque joueur à son salaire pour cette saison, je le place dans une trame de données temporaire, j'ajoute l'année à cette trame de données temporaire, puis j'ajoute cette trame de données temporaire à une trame de données principale qui inclut toutes les données que nous avons acquises. Le code final est ci-dessous !

```python
df = pd.DataFrame(columns=['Player','Salary','Year']) # creates master dataframe 

driver = webdriver.Chrome('/Users/MyUsername/Downloads/chromedriver')

for yr in range(1990,2019):
    page_num = str(yr) + '-' + str(yr+1) +'/'
    url = 'https://hoopshype.com/salaries/players/' + page_num
    driver.get(url)
    
    players = driver.find_elements_by_xpath('//td[@class="name"]')
    salaries = driver.find_elements_by_xpath('//td[@class="hh-salaries-sorted"]') 
    
    players_list = []
    for p in range(len(players)):
        players_list.append(players[p].text)
    
    salaries_list = []
    for s in range(len(salaries)):
        salaries_list.append(salaries[s].text)
    
    data_tuples = list(zip(players_list[1:],salaries_list[1:])) # list of each players name and salary paired together
    temp_df = pd.DataFrame(data_tuples, columns=['Player','Salary']) # creates dataframe of each tuple in list
    temp_df['Year'] = yr # adds season beginning year to each dataframe
    df = df.append(temp_df) # appends to master dataframe
    
driver.close()
```

#### Voici un code mis à jour avec les modifications de version

```python
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

chromedriver_path = '../chromedriver-mac-x64/chromedriver'
chrome_options = Options()
chrome_options.add_argument("--disable-search-engine-choice-screen")
driver = webdriver.Chrome(options=chrome_options)

driver.get("https://olympics.com/fr/paris-2024/medailles")

element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
)
element.click()

data = []

def scroll_and_collect():
    last_scroll_position = 0
    scroll_step = 300

    while True:
        elements = driver.find_elements(By.CSS_SELECTOR, "div[data-index]")

        for element in elements:
            try:
                span_name = element.find_element(By.CLASS_NAME, 'euzfwma5.emotion-srm-uu3d5n')
                row = [span_name.text]

                spans_81g9w1 = element.find_elements(By.CLASS_NAME, 'e1oix8v91.emotion-srm-81g9w1')
                if len(spans_81g9w1) == 3:
                    row.extend([span.text for span in spans_81g9w1])
                else:
                    continue

                span_5nhv3o = element.find_element(By.CLASS_NAME, 'e1oix8v91.emotion-srm-5nhv3o')
                row.append(span_5nhv3o.text)

                data.append(row)
            except:
                continue

        driver.execute_script(f"window.scrollBy(0, {scroll_step});")
        time.sleep(1)
        
        new_scroll_position = driver.execute_script("return window.pageYOffset;")
        if new_scroll_position == last_scroll_position:
            break
        last_scroll_position = new_scroll_position

scroll_and_collect()

tab = []
for row in data:
    tab.append(row)
    
df = pd.DataFrame(data, columns=["Pays", "Or", "Argent", "Bronze", "Total"])
df = df.drop_duplicates(subset='Pays')

driver.quit()
```

```python
df.to_csv('Olympics.csv', index=False)
```